<a href="https://colab.research.google.com/github/Malik-Raheel/data_analysis/blob/main/working_strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas_ta
!pip install ccxt
!pip install pandas
!pip install numpy
!pip install ta
!pip install scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=f433c519ae6af110374aaae43a1be9687c3c257903140738392720b61cb282c5
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.6/288.6 kB 13.9 MB/s eta 0:00:00
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.10
    Uninstalling aiohttp-3.11.10:
      Successfully uninstalled aiohttp-3.11.10
  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl s

In [2]:
import ccxt
import pandas as pd
import numpy as np
from ta.momentum import RSIIndicator
from ta.volatility import AverageTrueRange

# Initialize the exchange
exchange = ccxt.kucoin()  # You can replace 'kucoin' with your preferred exchange

# Symbols to analyze
symbols = ["BTC/USDT", "ETH/USDT", "AVAX/USDT", "SOL/USDT", "XAVA/USDT", "SLIM/USDT", "HNT/USDT", "NEAR/USDT", "DOGE/USDT"]

# Function to fetch historical data
def fetch_ccxt_data(symbol, timeframe, limit=500):
    """Fetch historical OHLCV data using CCXT."""
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
    df = pd.DataFrame(ohlcv, columns=["time", "open", "high", "low", "close", "volume"])
    df["time"] = pd.to_datetime(df["time"], unit="ms")
    return df

# Function to calculate indicators
def calculate_indicators(df, rsi_period=14, atr_period=14):
    """
    Calculate RSI and ATR indicators for the given dataframe.
    - RSI (Relative Strength Index) helps detect overbought or oversold conditions.
    - ATR (Average True Range) measures volatility.
    """
    df["RSI"] = RSIIndicator(df["close"], window=rsi_period).rsi()
    atr = AverageTrueRange(df["high"], df["low"], df["close"], window=atr_period)
    df["ATR"] = atr.average_true_range()
    return df

# Function to generate trading signals
def generate_signals(symbol, data_1d, data_4h, data_15m, data_weekly):
    """Generate buy/sell signals based on technical indicators."""
    signals = []

    # Weekly chart for long-term trend
    if data_weekly["RSI"].iloc[-1] > 90:
        trend = "Overbought"
    elif data_weekly["RSI"].iloc[-1] < 40:
        trend = "Oversold"
    else:
        trend = "Neutral"

    # Daily chart (1D) signals
    if data_1d["RSI"].iloc[-1] < 40:
        signals.append({
            "symbol": symbol,
            "timeframe": "1D",
            "action": "BUY",
            "price": data_1d["close"].iloc[-1],
            "reason": "Oversold on 1D RSI",
            "trend": trend
        })
    elif data_1d["RSI"].iloc[-1] > 80:
        signals.append({
            "symbol": symbol,
            "timeframe": "1D",
            "action": "SELL",
            "price": data_1d["close"].iloc[-1],
            "reason": "Overbought on 1D RSI",
            "trend": trend
        })

    # 4-hour chart (4H) signals
    if data_4h["RSI"].iloc[-1] < 40 and data_4h["close"].iloc[-1] > data_4h["ATR"].iloc[-1]:
        signals.append({
            "symbol": symbol,
            "timeframe": "4H",
            "action": "BUY",
            "price": data_4h["close"].iloc[-1],
            "reason": "Bullish RSI with strong support on 4H",
            "trend": trend
        })
    elif data_4h["RSI"].iloc[-1] > 85 and data_4h["close"].iloc[-1] < data_4h["ATR"].iloc[-1]:
        signals.append({
            "symbol": symbol,
            "timeframe": "4H",
            "action": "SELL",
            "price": data_4h["close"].iloc[-1],
            "reason": "Bearish RSI with weak resistance on 4H",
            "trend": trend
        })

    # 15-minute chart (15M) signals for intraday trades
    if data_15m["RSI"].iloc[-1] < 40 and data_15m["close"].iloc[-1] > data_15m["ATR"].iloc[-1]:
        signals.append({
            "symbol": symbol,
            "timeframe": "15M",
            "action": "BUY",
            "price": data_15m["close"].iloc[-1],
            "reason": "Bullish RSI and ATR support on 15M",
            "trend": trend
        })
    elif data_15m["RSI"].iloc[-1] > 90 and data_15m["close"].iloc[-1] < data_15m["ATR"].iloc[-1]:
        signals.append({
            "symbol": symbol,
            "timeframe": "15M",
            "action": "SELL",
            "price": data_15m["close"].iloc[-1],
            "reason": "Overbought RSI and ATR resistance on 15M",
            "trend": trend
        })

    return signals

# Main execution
all_signals = []

for symbol in symbols:
    try:
        # Fetch data for each timeframe
        data_1d = fetch_ccxt_data(symbol, "1d")
        data_4h = fetch_ccxt_data(symbol, "4h")
        data_15m = fetch_ccxt_data(symbol, "15m")
        data_weekly = fetch_ccxt_data(symbol, "1w")

        # Calculate indicators for each timeframe
        data_1d = calculate_indicators(data_1d)
        data_4h = calculate_indicators(data_4h)
        data_15m = calculate_indicators(data_15m)
        data_weekly = calculate_indicators(data_weekly)

        # Generate signals
        signals = generate_signals(symbol, data_1d, data_4h, data_15m, data_weekly)
        all_signals.extend(signals)

    except Exception as e:
        # Suppress errors but continue processing other symbols
        pass

# Output results (only signals, clean format)
for signal in all_signals:
    print(f"{signal['symbol']} {signal['timeframe']} Signal: {signal['action']} at {signal['price']:.2f} "
          f"(Reason: {signal['reason']}, Trend: {signal['trend']})")


HNT/USDT 1D Signal: BUY at 4.91 (Reason: Oversold on 1D RSI, Trend: Neutral)
